# Sherlock

In [ ]:
#!wget https://www.gutenberg.org/files/1661/1661-0.txt > sherlock.txt

In [ ]:
!pip install -qU sentence-transformers bioinfokit mplcursors

In [ ]:
from nltk import sent_tokenize, word_tokenize
import nltk

nltk.download('punkt')

In [ ]:
f = open('sherlock.txt', 'r')
contents = f.read().replace('\n', ' ')
print(contents[:2000])

In [ ]:
sent_text = sent_tokenize(contents)
print(sent_text[:30])
print([s for s in sent_text if 'X.' in s])

# for sentence in sent_text:
#     tokenized_text = nltk.word_tokenize(sent_text)
# tagged = nltk.pos_tag(tokenized_text)
# print(tagged)
# print(tokenized_text[:40])

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = model.encode(sent_text)

In [ ]:
from sklearn.manifold import TSNE
tsne_emb = TSNE(n_components=2, perplexity=30.0, n_iter=1000, verbose=1).fit_transform(sentence_embeddings)

In [ ]:
print(len(sent_text))
print(sentence_embeddings.shape)
print(tsne_emb.shape)

In [ ]:
import seaborn as sns

colors = sns.color_palette("Paired", 16)
print(colors)
print('Num colors:', len(colors))

In [ ]:
import re

sents_by_chapter = []
ch = 0
new_ch = False
ch_sent = []

data = []
for index, s in enumerate(sent_text):
#     print(ch)
    color = colors[ch]
    label = "Ch. {} (n={})".format(str(ch).zfill(2), str(index).zfill(3))
    data.append((tsne_emb[index], ch, sent_text[index], color, label))
    new_ch = False
    if re.match('(.*( ){2,}[IVX]{1,}\.)|(^[IVX]{1,}\.)', s):
        print(s)
#         sents_by_chapter.append(ch_sent)
        new_ch = True
        ch += 1

# for s in sent_text:
#     if new_ch:
#         ch_sent = []
#     ch_sent.append(s)
#     new_ch = False
#     if re.match('(.*( ){2,}[IVX]{1,}\.)|(^[IVX]{1,}\.)', s):
#         print(s)
#         sents_by_chapter.append(ch_sent)
#         new_ch = True
#         ch += 1
        
# sents_by_chapter.append(ch_sent)
# ch += 1
        
print('\n----\n')

print(data[13])
print(data[4420])
    
print('\n----\n')
# print(len(sents_by_chapter), 'Length of chapters list')
print(ch, 'Chapter index')
print(len(data))

In [ ]:
for c in sents_by_chapter:
    print(len(c))
ch_chunks = []
idx = 0
for c in sents_by_chapter:
    ch_chunks.append(idx)
    idx += len(c)
print(ch_chunks)
print(len(ch_chunks))
print(len(sents_by_chapter))

sum = 0
for i, a in enumerate(ch_chunks[:-1]):
    sum += ch_chunks[i+1] - ch_chunks[i]
print(sum)

In [ ]:
# https://www.reneshbedre.com/blog/tsne.html

# from bioinfokit.visuz import cluster
# cluster.tsneplot(score=tsne_emb, dotsize=1)
# # color_class = sentence_y
# # cluster.tsneplot(score=fashion_tsne, colorlist=color_class, legendpos='upper right', legendanchor=(1.15, 1) )

In [ ]:
%matplotlib notebook

# https://scipy-lectures.org/packages/scikit-learn/auto_examples/plot_tsne.html
# Alternative visualization using SKLearn, in color, segmented by chapter

from matplotlib import pyplot as plt
import numpy as np
import mplcursors
import pandas as pd
import textwrap

# data = []
# for a, b, c in some_function_that_yields_data():
#     data.append([a, b, c])

# df = pd.DataFrame(data, columns=['A', 'B', 'C'])

# print(sents_by_chapter[14][-1])

# data = []
# for c_idx, n in enumerate(ch_chunks[2:-1]):
#     print('Doing chapter {}'.format(c_idx))
# #     print(c_idx, n)
#     upper_bound = ch_chunks[2+c_idx+1]
# #     print(n, upper_bound)
#     print('Will get sentences {}-{}'.format(n, upper_bound))
# #     for i in range(n, upper_bound):
# #         s = sents_by_chapter[2+c_idx][i]
# #         print(s)

df = pd.DataFrame(data, columns=['tsne', 'ch', 'sent', 'color', 'label'])

# print(df.head())

fig, ax = plt.subplots()
fig.set_dpi(150)
# for c_idx, n in enumerate(ch_chunks[2:-1]):
#     upper_bound = ch_chunks[2+c_idx+1]
#     sents = tsne_emb[n+2:upper_bound]
#     color = np.array([colors[c_idx]])
#     num_sents = upper_bound - n
#     l = "Ch. {} (n={})".format(str(c_idx+1).zfill(2), str(num_sents).zfill(3))
#     sc = ax.scatter(*zip(*sents[2:]), c=color, label=l, s=4)
 
# df["color"] = df['ch'].apply(lambda x: colors[x])
# print(df['ch'])
# ax.scatter(df[xcol], df[ycol], c=df.Color)
# for index, row in df.iterrows():
# for group in df.groupby(['ch']):
# #     print(row)
# #     l = group[0]['label']
#     print(len(group))
sc = ax.scatter(*zip(*df['tsne']), c=df.color, s=2)#, label=df.label)

cursor = mplcursors.cursor(hover=True)
cursor.connect("add", lambda sel: sel.annotation.set_text(
    textwrap.fill(
        sel.annotation.get_text() + '\n' + df["sent"][sel.index], 20
    )
))

# l = df['label'].unique()
# print(l)
# print(sc.legend_elements()[0])

# plt.legend(handles=sc.legend_elements()[0], labels=l)

# ax.legend(bbox_to_anchor=(0.95, 0.7), markerscale=4, fontsize=6)
plt.savefig('tsne_chapters.png')
plt.show()